# 학번: 20901     이름: 경승민
## 책의 첫 문장으로 발행년도와 장르 예측하기

## 1. 데이터 입력

In [ ]:
# 파일 선택 창을 통해 파일을 불러오기
from google.colab import files
filename = list(files.upload().keys())[0]
print(f'업로드된 파일: {filename}')

In [ ]:
# 판다스 모듈 불러와서 파일 읽어 들이기
import pandas as pd
df = pd.read_csv(filename)
df

## 2. 데이터 탐색

In [ ]:
# 데이터 기초 정보 살펴보기
df.info()

In [ ]:
# 상단 10개 데이터 살펴보기
df.head(10)

In [ ]:
# 데이터 통계치 살펴보기
df.describe()

In [ ]:
# 결측치 확인
print('결측치 개수:')
print(df.isnull().sum())

## 3. 데이터 시각화

In [ ]:
# 데이터 시각화1: 장르별 책의 개수
import matplotlib.pyplot as plt

genre_counts = df['Genre'].value_counts()
print('장르별 책의 개수:')
print(genre_counts)

In [ ]:
# 장르별 개수를 막대그래프로 표현
plt.figure(figsize=(12, 6))
genre_counts.plot.bar()
plt.title('Genre Distribution')
plt.xlabel('Genre')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# 데이터 시각화2: 년도별 책의 개수
year_counts = df['Year'].value_counts().sort_index()
print('년도별 책의 개수 (상위 10개):')
print(year_counts.head(10))

In [ ]:
# 년도별 분포를 히스토그램으로 표현
plt.figure(figsize=(12, 6))
plt.hist(df['Year'], bins=50, edgecolor='black')
plt.title('Year Distribution')
plt.xlabel('Year')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
# 데이터 시각화3: 문장 길이 분석
df['sentence_length'] = df['Sentence'].apply(lambda x: len(str(x)))
print('문장 길이 통계:')
print(df['sentence_length'].describe())

In [ ]:
# 문장 길이 분포를 히스토그램으로 표현
plt.figure(figsize=(12, 6))
plt.hist(df['sentence_length'], bins=50, edgecolor='black')
plt.title('Sentence Length Distribution')
plt.xlabel('Sentence Length')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## 4. 데이터 전처리

In [ ]:
# 결측치 제거
df_clean = df.dropna()
print(f'전처리 전 데이터 개수: {len(df)}')
print(f'전처리 후 데이터 개수: {len(df_clean)}')

In [ ]:
# 텍스트 데이터를 숫자 벡터로 변환 (TF-IDF 사용)
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터라이저 생성 (상위 1000개 단어만 사용)
tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
X_tfidf = tfidf.fit_transform(df_clean['Sentence'])

print(f'변환된 데이터 형태: {X_tfidf.shape}')
print(f'특성(단어) 개수: {X_tfidf.shape[1]}')

In [ ]:
# 장르를 숫자로 인코딩
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_genre = label_encoder.fit_transform(df_clean['Genre'])

print('장르 레이블 인코딩:')
for i, genre in enumerate(label_encoder.classes_):
    print(f'{i}: {genre}')

In [ ]:
# 년도 데이터 준비 (년도는 그대로 사용)
y_year = df_clean['Year'].values
print(f'년도 범위: {y_year.min()} ~ {y_year.max()}')

## 5. 데이터 분리 (훈련 데이터와 테스트 데이터)

In [ ]:
# 훈련 데이터와 테스트 데이터 나누기 (7:3 비율)
from sklearn.model_selection import train_test_split

X_train, X_test, y_genre_train, y_genre_test, y_year_train, y_year_test = train_test_split(
    X_tfidf, y_genre, y_year, test_size=0.3, random_state=42
)

print(f'훈련 데이터 크기: {X_train.shape}')
print(f'테스트 데이터 크기: {X_test.shape}')

## 6. 모델 학습 - 장르 예측

In [ ]:
# 로지스틱 회귀 모델로 장르 예측
from sklearn.linear_model import LogisticRegression

genre_model = LogisticRegression(max_iter=1000, random_state=42)
genre_model.fit(X_train, y_genre_train)
print('장르 예측 모델 학습 완료')

In [ ]:
# 장르 예측 모델 평가
from sklearn.metrics import accuracy_score

genre_pred = genre_model.predict(X_test)
genre_accuracy = accuracy_score(y_genre_test, genre_pred)

print(f'장르 예측 정확도: {genre_accuracy * 100:.2f}%')

In [ ]:
# 추가 모델 시도: 랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier

genre_rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
genre_rf_model.fit(X_train, y_genre_train)

genre_rf_pred = genre_rf_model.predict(X_test)
genre_rf_accuracy = accuracy_score(y_genre_test, genre_rf_pred)

print(f'장르 예측 정확도 (랜덤 포레스트): {genre_rf_accuracy * 100:.2f}%')

## 7. 모델 학습 - 년도 예측

In [ ]:
# 선형 회귀 모델로 년도 예측
from sklearn.linear_model import LinearRegression

year_model = LinearRegression()
year_model.fit(X_train, y_year_train)
print('년도 예측 모델 학습 완료')

In [ ]:
# 년도 예측 모델 평가
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

year_pred = year_model.predict(X_test)
year_mse = mean_squared_error(y_year_test, year_pred)
year_r2 = r2_score(y_year_test, year_pred)
year_mae = np.mean(np.abs(y_year_test - year_pred))

print(f'년도 예측 MSE: {year_mse:.2f}')
print(f'년도 예측 R2 Score: {year_r2:.2f}')
print(f'년도 예측 평균 오차: {year_mae:.2f}년')

In [ ]:
# 년도 예측 정확도 계산 (±20년 이내를 정확한 예측으로 간주)
tolerance = 20
year_accuracy = np.mean(np.abs(y_year_test - year_pred) <= tolerance)
print(f'년도 예측 정확도 (±{tolerance}년 이내): {year_accuracy * 100:.2f}%')

In [ ]:
# 추가 모델 시도: 랜덤 포레스트 회귀
from sklearn.ensemble import RandomForestRegressor

year_rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
year_rf_model.fit(X_train, y_year_train)

year_rf_pred = year_rf_model.predict(X_test)
year_rf_mse = mean_squared_error(y_year_test, year_rf_pred)
year_rf_r2 = r2_score(y_year_test, year_rf_pred)
year_rf_mae = np.mean(np.abs(y_year_test - year_rf_pred))
year_rf_accuracy = np.mean(np.abs(y_year_test - year_rf_pred) <= tolerance)

print(f'년도 예측 MSE (랜덤 포레스트): {year_rf_mse:.2f}')
print(f'년도 예측 R2 Score (랜덤 포레스트): {year_rf_r2:.2f}')
print(f'년도 예측 평균 오차 (랜덤 포레스트): {year_rf_mae:.2f}년')
print(f'년도 예측 정확도 (랜덤 포레스트, ±{tolerance}년 이내): {year_rf_accuracy * 100:.2f}%')

## 8. 모델 평가 시각화

In [ ]:
# 장르 예측 성능 비교 막대그래프
import matplotlib.pyplot as plt

models = ['Logistic Regression', 'Random Forest']
accuracies = [genre_accuracy * 100, genre_rf_accuracy * 100]

plt.figure(figsize=(10, 6))
plt.bar(models, accuracies)
plt.title('Genre Prediction Model Comparison')
plt.ylabel('Accuracy (%)')
plt.ylim(0, 100)
for i, v in enumerate(accuracies):
    plt.text(i, v + 2, f'{v:.2f}%', ha='center', va='bottom')
plt.tight_layout()
plt.show()

In [ ]:
# 년도 예측 성능 비교
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 선형 회귀 산점도
axes[0].scatter(y_year_test, year_pred, alpha=0.5)
axes[0].plot([y_year_test.min(), y_year_test.max()], [y_year_test.min(), y_year_test.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Year')
axes[0].set_ylabel('Predicted Year')
axes[0].set_title('Linear Regression: Year Prediction')

# 랜덤 포레스트 산점도
axes[1].scatter(y_year_test, year_rf_pred, alpha=0.5)
axes[1].plot([y_year_test.min(), y_year_test.max()], [y_year_test.min(), y_year_test.max()], 'r--', lw=2)
axes[1].set_xlabel('Actual Year')
axes[1].set_ylabel('Predicted Year')
axes[1].set_title('Random Forest: Year Prediction')

plt.tight_layout()
plt.show()

## 9. 새로운 데이터로 예측하기

In [ ]:
# 새로운 책의 첫 문장으로 예측하기
new_sentence = "It was the best of times, it was the worst of times."

# 텍스트를 TF-IDF 벡터로 변환
new_sentence_tfidf = tfidf.transform([new_sentence])

# 장르 예측 (더 좋은 모델 사용)
if genre_rf_accuracy > genre_accuracy:
    predicted_genre_idx = genre_rf_model.predict(new_sentence_tfidf)[0]
    predicted_genre_proba = genre_rf_model.predict_proba(new_sentence_tfidf)[0]
    model_used = '랜덤 포레스트'
else:
    predicted_genre_idx = genre_model.predict(new_sentence_tfidf)[0]
    predicted_genre_proba = genre_model.predict_proba(new_sentence_tfidf)[0]
    model_used = '로지스틱 회귀'

predicted_genre = label_encoder.inverse_transform([predicted_genre_idx])[0]

# 년도 예측 (더 좋은 모델 사용)
if year_rf_accuracy > year_accuracy:
    predicted_year = year_rf_model.predict(new_sentence_tfidf)[0]
else:
    predicted_year = year_model.predict(new_sentence_tfidf)[0]

print(f'입력 문장: "{new_sentence}"')
print(f'예측 장르: {predicted_genre} (사용 모델: {model_used})')
print(f'예측 년도: {predicted_year:.0f}년')
print(f'장르 예측 확률: {max(predicted_genre_proba) * 100:.2f}%')

In [ ]:
# 여러 개의 새로운 문장 예측해보기
test_sentences = [
    "The detective walked into the dark alley, his gun drawn.",
    "Once upon a time, in a land far away, there lived a princess.",
    "The spaceship landed on the red planet, and the crew prepared for exploration.",
    "I was born in a small town in 1950, where life was simple."
]

print('여러 문장 예측 결과:\n')
for i, sentence in enumerate(test_sentences, 1):
    sentence_tfidf = tfidf.transform([sentence])
    
    # 더 좋은 모델 사용
    if genre_rf_accuracy > genre_accuracy:
        pred_genre_idx = genre_rf_model.predict(sentence_tfidf)[0]
    else:
        pred_genre_idx = genre_model.predict(sentence_tfidf)[0]
    
    if year_rf_accuracy > year_accuracy:
        pred_year = year_rf_model.predict(sentence_tfidf)[0]
    else:
        pred_year = year_model.predict(sentence_tfidf)[0]
    
    pred_genre = label_encoder.inverse_transform([pred_genre_idx])[0]
    
    print(f'{i}. 문장: "{sentence}"')
    print(f'   예측 장르: {pred_genre}, 예측 년도: {pred_year:.0f}년\n')

## 10. 모델 성능 요약

In [ ]:
# 최종 모델 성능 요약
print('='*60)
print('모델 성능 요약')
print('='*60)
print(f'\n[장르 예측 모델]')
print(f'로지스틱 회귀 정확도: {genre_accuracy * 100:.2f}%')
print(f'랜덤 포레스트 정확도: {genre_rf_accuracy * 100:.2f}%')
print(f'최고 성능 모델: {"랜덤 포레스트" if genre_rf_accuracy > genre_accuracy else "로지스틱 회귀"}')

print(f'\n[년도 예측 모델]')
print(f'선형 회귀 정확도 (±{tolerance}년): {year_accuracy * 100:.2f}%')
print(f'선형 회귀 평균 오차: {year_mae:.2f}년')
print(f'랜덤 포레스트 정확도 (±{tolerance}년): {year_rf_accuracy * 100:.2f}%')
print(f'랜덤 포레스트 평균 오차: {year_rf_mae:.2f}년')
print(f'최고 성능 모델: {"랜덤 포레스트" if year_rf_accuracy > year_accuracy else "선형 회귀"}')

# 목표 달성 여부
best_genre_acc = max(genre_accuracy, genre_rf_accuracy) * 100
best_year_acc = max(year_accuracy, year_rf_accuracy) * 100
overall_acc = (best_genre_acc + best_year_acc) / 2

print(f'\n[종합 성능]')
print(f'장르 예측 최고 정확도: {best_genre_acc:.2f}%')
print(f'년도 예측 최고 정확도: {best_year_acc:.2f}%')
print(f'전체 평균 정확도: {overall_acc:.2f}%')
print(f'\n성능 목표 (60% 이상): {"달성 ✓" if overall_acc >= 60 else "미달성 ✗"}')
print('='*60)

## 11. 대화형 예측 시스템

사용자가 직접 문장을 입력하면 장르와 년도를 예측해주는 시스템입니다.

In [ ]:
# 예측 함수 정의
def predict_book_info(sentence):
    """
    입력된 문장으로 장르와 년도를 예측하는 함수
    
    Args:
        sentence (str): 예측할 문장
    
    Returns:
        dict: 예측 결과를 담은 딕셔너리
    """
    # 텍스트를 TF-IDF 벡터로 변환
    sentence_tfidf = tfidf.transform([sentence])
    
    # 최고 성능 모델 선택하여 장르 예측
    if genre_rf_accuracy > genre_accuracy:
        pred_genre_idx = genre_rf_model.predict(sentence_tfidf)[0]
        pred_genre_proba = genre_rf_model.predict_proba(sentence_tfidf)[0]
        genre_model_name = '랜덤 포레스트'
    else:
        pred_genre_idx = genre_model.predict(sentence_tfidf)[0]
        pred_genre_proba = genre_model.predict_proba(sentence_tfidf)[0]
        genre_model_name = '로지스틱 회귀'
    
    pred_genre = label_encoder.inverse_transform([pred_genre_idx])[0]
    genre_confidence = max(pred_genre_proba) * 100
    
    # 최고 성능 모델 선택하여 년도 예측
    if year_rf_accuracy > year_accuracy:
        pred_year = year_rf_model.predict(sentence_tfidf)[0]
        year_model_name = '랜덤 포레스트'
    else:
        pred_year = year_model.predict(sentence_tfidf)[0]
        year_model_name = '선형 회귀'
    
    # 년도를 시대로 변환
    if pred_year < 1900:
        era = '19세기 후반'
    elif pred_year < 1950:
        era = '20세기 전반 (1900-1949)'
    elif pred_year < 1980:
        era = '20세기 중반 (1950-1979)'
    elif pred_year < 2000:
        era = '20세기 후반 (1980-1999)'
    else:
        era = '21세기 (2000년 이후)'
    
    return {
        'genre': pred_genre,
        'genre_confidence': genre_confidence,
        'genre_model': genre_model_name,
        'year': int(pred_year),
        'era': era,
        'year_model': year_model_name
    }

print('예측 함수 준비 완료!')

In [ ]:
# 대화형 예측 시스템
print('='*70)
print('책 문장 예측 시스템에 오신 것을 환영합니다!')
print('='*70)
print('\n이 시스템은 책의 첫 문장을 분석하여:')
print('  1. 어떤 장르의 책인지')
print('  2. 어느 시대에 쓰여진 책인지')
print('를 예측해드립니다.\n')
print('종료하려면 "종료", "quit", "exit" 중 하나를 입력하세요.')
print('='*70)

while True:
    print('\n' + '-'*70)
    user_input = input('\n문장을 입력하세요: ').strip()
    
    # 종료 조건
    if user_input.lower() in ['종료', 'quit', 'exit', 'q']:
        print('\n시스템을 종료합니다. 감사합니다!')
        break
    
    # 빈 입력 체크
    if not user_input:
        print('⚠️  문장을 입력해주세요.')
        continue
    
    # 너무 짧은 입력 체크
    if len(user_input.split()) < 3:
        print('⚠️  최소 3단어 이상의 문장을 입력해주세요.')
        continue
    
    try:
        # 예측 수행
        result = predict_book_info(user_input)
        
        # 결과 출력
        print('\n' + '='*70)
        print('📖 예측 결과')
        print('='*70)
        print(f'\n입력 문장: "{user_input}"\n')
        print(f'📚 장르: {result["genre"]}')
        print(f'   - 신뢰도: {result["genre_confidence"]:.2f}%')
        print(f'   - 사용 모델: {result["genre_model"]}')
        print(f'\n📅 예측 년도: {result["year"]}년')
        print(f'   - 시대 구분: {result["era"]}')
        print(f'   - 사용 모델: {result["year_model"]}')
        print('\n' + '='*70)
        
    except Exception as e:
        print(f'\n⚠️  예측 중 오류가 발생했습니다: {str(e)}')
        print('다른 문장을 입력해주세요.')

### 예측 시스템 사용 예시

위의 셀을 실행하면 대화형 시스템이 시작됩니다. 다음과 같이 사용할 수 있습니다:

**예시 입력:**
```
The sun was setting over the dusty plains as the cowboy rode into town.
```

**예상 출력:**
```
📚 장르: Cowboy/Western Tales
📅 예측 년도: 1965년 (20세기 중반)
```

**다른 예시:**
- "In the beginning, God created the heavens and the earth." → Philosophy & Religion, 1900-1950년대
- "The body was found at midnight in the mansion's library." → Crime/Detective, 1960-1990년대
- "She opened her laptop and started coding the neural network." → Science & Technology, 2000년 이후

In [ ]:
# 단일 문장 빠른 예측 (반복 입력 없이 한 번만 예측)
def quick_predict(sentence):
    """
    한 번에 하나의 문장만 빠르게 예측하는 함수
    """
    result = predict_book_info(sentence)
    print('\n' + '='*70)
    print(f'입력: "{sentence}"')
    print('-'*70)
    print(f'장르: {result["genre"]} (신뢰도: {result["genre_confidence"]:.1f}%)')
    print(f'년도: {result["year"]}년 ({result["era"]})')
    print('='*70)

# 사용 예시
print('빠른 예측 함수 사용 예시:\n')
quick_predict("Call me Ishmael.")
quick_predict("In a hole in the ground there lived a hobbit.")
quick_predict("Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal.")